In [1]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
vehicle_classes = [2, 3, 5, 7]  #'car', 'motorbike', 'bus', 'truck'

Using cache found in C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-12 Python-3.12.5 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
import pandas as pd
import cv2
import random
import time
from collections import Counter
from pathlib import Path

dataset_dir = Path('vehicle_dataset')
dataset_dir.mkdir(exist_ok=True)


vehicle_classes = {
    2: 'Cars',
    3: 'Motorbikes',
    5: 'Buses',
    7: 'Trucks'
}


def get_latest_vehicle_count(directory):
    csv_files = [f for f in directory.glob("*.csv") if f.is_file()]
    if not csv_files:
        return 0
    csv_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)  
    latest_csv = csv_files[0] 
    df = pd.read_csv(latest_csv)
    return df['Total'].iloc[0]  


def save_vehicle_info(vehicle_count_by_class, frame_id, confidences):
    data = {
        'Cars': [vehicle_count_by_class['Cars']],
        'Motorbikes': [vehicle_count_by_class['Motorbikes']],
        'Buses': [vehicle_count_by_class['Buses']],
        'Trucks': [vehicle_count_by_class['Trucks']],
        'Total': [sum(vehicle_count_by_class.values())],
        'Confidence': [max(confidences) if confidences else 0]
    }
    df = pd.DataFrame(data)
    df.to_csv(dataset_dir / f'vehicles_frame_{frame_id}.csv', index=False)
    print(f"Saved vehicle data for frame {frame_id}.")

def keep_latest_20_csv(directory):
    csv_files = [f for f in directory.glob("*.csv") if f.is_file()]
    csv_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
    if len(csv_files) > 20:
        files_to_delete = csv_files[20:]
        for file in files_to_delete:
            file.unlink()
            print(f"Deleted: {file}")

def get_most_frequent_count(counts):
    count_freq = Counter(counts)
    most_frequent = count_freq.most_common()
    max_frequency = most_frequent[0][1]
    most_frequent_counts = [count for count, freq in most_frequent if freq == max_frequency]
    return max(most_frequent_counts)

def traffic_light_system(cap):
    signal_2_cars = random.randint(0, 30)
    print(f"Initial cars at Signal 2: {signal_2_cars}")
    
    signal_1_green = False
    signal_2_green = False
    signal_1_cars = 0 

    frame_id = 0
    count_window = []

    if signal_1_cars < signal_2_cars:
        signal_1_green = False
        signal_2_green = True
    else:
        signal_1_green = True
        signal_2_green = False

    last_signal_2_decrease_time = time.time() 

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

       
        results = model(frame) 
        detections = results.pandas().xyxy[0]

        vehicle_detections = detections[detections['class'].isin(vehicle_classes.keys())]

        vehicle_count_by_class = {name: 0 for name in vehicle_classes.values()}
        confidences = []

        for idx, row in vehicle_detections.iterrows():
            vehicle_name = vehicle_classes[int(row['class'])]
            vehicle_count_by_class[vehicle_name] += 1
            confidences.append(row['confidence'])

        total_vehicle_count = sum(vehicle_count_by_class.values())
        count_window.append(total_vehicle_count)

        if len(count_window) > 10:
            count_window.pop(0)

        
        if frame_id % 60 == 0 and len(count_window) == 10:
            most_frequent_count = get_most_frequent_count(count_window)
            save_vehicle_info(vehicle_count_by_class, frame_id, confidences)
            keep_latest_20_csv(dataset_dir)

            
            signal_1_cars = get_latest_vehicle_count(dataset_dir)

            
            if signal_1_green and signal_2_green:
                print("Both signals cannot be green simultaneously. Please check the logic.")

            
            if signal_1_green:
                if (signal_1_cars < signal_2_cars - 5) or (signal_2_cars >= signal_1_cars):
                    print("Switching to Signal 2.")
                    signal_1_green = False
                    signal_2_green = True  
                else:
                    print("Signal 1 remains green.")

            elif signal_2_green:
                if (signal_2_cars < signal_1_cars - 5) or (signal_1_cars >= signal_2_cars):
                    print("Switching to Signal 1.")
                    signal_2_green = False
                    signal_1_green = True  
                else:
                    print("Signal 2 remains green.")

        
        if signal_1_green:
            cv2.putText(frame, "Signal 1: GREEN", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Signal 1: RED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if signal_2_green:
            cv2.putText(frame, "Signal 2: GREEN", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Signal 2: RED", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.putText(frame, f"Signal 1 Vehicles: {signal_1_cars}", (frame.shape[1] - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, f"Signal 2 Vehicles: {signal_2_cars}", (frame.shape[1] - 200, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        cv2.imshow('Vehicle Detection and Traffic Light System', frame)

        current_time = time.time()
        if signal_2_green and signal_2_cars > 0:
            if current_time - last_signal_2_decrease_time >= 2:  
                signal_2_cars -= 1  
                last_signal_2_decrease_time = current_time  
                print(f"Signal 2 car count decreased: {signal_2_cars}")

        if signal_2_cars <= 5:
            print(f"Signal 2 is near zero: {signal_2_cars}. Checking to switch signals.")
            if signal_1_cars > signal_2_cars:  
                signal_2_green = False
                signal_1_green = True 
                signal_1_cars = random.randint(0, 30) 
                print(f"New random cars at Signal 1: {signal_1_cars}")
            signal_2_cars = random.randint(0, 30) 

        frame_id += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

cap = cv2.VideoCapture(0)
traffic_light_system(cap)


Initial cars at Signal 2: 24


C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models

Signal 2 car count decreased: 23


C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models

Signal 2 car count decreased: 22


...

C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Signal 2 car count decreased: 25


C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models

Signal 2 car count decreased: 24


C:\Users\ahaan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
